<div style="border:solid green 2px; padding: 20px">
<b>Привет, Евгений!</b>

Меня зовут Алан, и я буду проверять твой проект. Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Тобой уже проделана большая работа над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет прният. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания, либо рекоммендации к улучшению. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [183]:
import pandas as pd
main_table = pd.read_csv("/datasets/data.csv")
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
import numpy as np

In [184]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

solvency_table = main_table[['children', 'family_status', 'family_status_id', 'debt', 'total_income', 'purpose']]
main_table.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<div class="alert alert-warning">

**Комментарий ревьюера:** Библиотеки, лучше стараться импортировать в отдельной ячейке для ясности и удобства. Например, если они у тебя были в одной ячейке с загрузкой файла, то тебе при добавлении новой библиотеки в ходе работы больше не придется по новой загружать файл.
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Перенёс импортирование библиотек в новую ячейку</div>

In [185]:
main_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В исходной таблице присутствуют столбцы, которые не влияют на результаты исследования. К этим столбцам относятся:
- days_employed
- dob_years
- education
- gender
- income_type

Среди данных столбцов обнаружены артефакты, такие как отрицательный стаж работы, стаж работы превышающий сто лет. Эти и другие неясные вещи при необходимости можно было бы уточнить у того отдела, который составил данную таблицу.
На основе исхдной таблицы было составлена новая, только с теми столбцами, которые могут пригодится в исследовании.

В общей информации о таблице видно, что есть пропуски данных в столбцах days_employed и total_income. Также, в информации о таблице виден тип данных каждого столбца.

<div class="alert alert-danger">

**Комментарий ревьюера:** Все делаешь правильно, только прошу тебя также вывести общую информацию по таблице (типы данных, есть ли пропуски и т.д.)
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Вывел общую информацию о таблице. </div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [186]:
pd.options.mode.chained_assignment = None  
# убрал ошибку, которая не останавливала выволнение кода, но визуально выглядело некрасиво

In [187]:
solvency_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 6 columns):
children            21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 1009.1+ KB


In [188]:
total_income_median = solvency_table['total_income'].median()
solvency_table['total_income'] = solvency_table['total_income'].fillna(total_income_median)

**Вывод**

В столбце 'total_income' было около 10% пропусков. Было решено, что данный процент слишком велик, чтобы удалять строчки с пустым значением. Пустые значения были заменены медианой, поскольку, на мой взгляд, таким образом удалось избежать смещения баланса значений столбца.

Пропуски могли появится из-за ошибки, при обработки бланка (для получения кредита), из-за ошибки загрузки/выгрузки данных бланка на/из сервер(а) или изначально в бланке было не заполнено определенное поле.

### Замена типа данных

In [189]:
solvency_table['children'] = solvency_table['children'].abs()
solvency_table['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [192]:
solvency_table['total_income'] = solvency_table['total_income'].astype(np.int64)
solvency_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 6 columns):
children            21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(4), object(2)
memory usage: 1009.1+ KB


**Вывод**

В столбце 'children' были обнаружены отрицательные значения, чего быть не может. Поэтому значение для столбца было взято по модулю. Также бросается в глаза резкое увеличение значений столбца после значения 5. Возможно, это является артефактом. Но также может быть, что это следует трактовать как "более 5 детей". Это можно было бы уточнить у отдела, составлявшего исходную таблицу.

В столбце 'total_income' заменил вещественный тип данных на целочисленнный. Это было сделано для того, чтобы визуально сделать информацию в таблице более понятной.

<div class="alert alert-danger">

**Комментарий ревьюера:** На данном этапе попрошу тебя изменить вещественный тип данных в изучаемых столбцах на целочисленный.
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Изменил вещественный тип данных на целочисленный</div>

### Обработка дубликатов

In [193]:
solvency_table.duplicated().sum()
solvency_table = solvency_table.drop_duplicates()
solvency_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19832 entries, 0 to 21524
Data columns (total 6 columns):
children            19832 non-null int64
family_status       19832 non-null object
family_status_id    19832 non-null int64
debt                19832 non-null int64
total_income        19832 non-null int64
purpose             19832 non-null object
dtypes: int64(4), object(2)
memory usage: 1.1+ MB


In [194]:
solvency_table = solvency_table.reset_index()
del solvency_table['index']
solvency_table.info()
solvency_table.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19832 entries, 0 to 19831
Data columns (total 6 columns):
children            19832 non-null int64
family_status       19832 non-null object
family_status_id    19832 non-null int64
debt                19832 non-null int64
total_income        19832 non-null int64
purpose             19832 non-null object
dtypes: int64(4), object(2)
memory usage: 929.8+ KB


,children,family_status,family_status_id,debt,total_income,purpose
0,1,женат / замужем,0,0,253875,покупка жилья
1,1,женат / замужем,0,0,112080,приобретение автомобиля
2,0,женат / замужем,0,0,145885,покупка жилья
3,3,женат / замужем,0,0,267628,дополнительное образование
4,0,гражданский брак,1,0,158616,сыграть свадьбу


**Вывод**

Методом .duplicated().sum() было обнаружено 1680 дубликатов сток. Для избежания искажения результатов исследования, дубликаты были удалены методом drop_duplicates(). Индексы были обновлены методом reset_index(). Поскольку при использовании метода reset_index() с аргументом drop = True вылезала ошибка, пришлось применить метод без аргумента, и затем удалить столбец со старыми индексами.

На мой взгляд, появление дубликатов может быть обусловлено ошибками в процессе хранения банных на сервере.

### Лемматизация

In [195]:
line = -1 #Переменная, которая показывает, какая стока обрабатывается в текущий момент
for element in solvency_table['purpose']:
    text = element
    line += 1 #Каждый раз, когда цикл обрабатывает новую стоку, переменная увеличивается на 1
    lemmas = m.lemmatize(text)
    if 'автомобиль' in lemmas:
        solvency_table['purpose'][line] = 'автомобиль' #Тут и пригождается переменная line
    elif 'свадьба' in lemmas:
        solvency_table['purpose'][line] = 'свадьба'
    elif 'образование' in lemmas:
        solvency_table['purpose'][line] = 'образование'
    elif 'жильё' in lemmas:
         solvency_table['purpose'][line] = 'недвижимость'
    elif 'жилье' in lemmas:
         solvency_table['purpose'][line] = 'недвижимость'
    elif 'недвижимость' in lemmas:
         solvency_table['purpose'][line] = 'недвижимость'

In [196]:
solvency_table.head(15)

,children,family_status,family_status_id,debt,total_income,purpose
0,1,женат / замужем,0,0,253875,недвижимость
1,1,женат / замужем,0,0,112080,автомобиль
2,0,женат / замужем,0,0,145885,недвижимость
3,3,женат / замужем,0,0,267628,образование
4,0,гражданский брак,1,0,158616,свадьба
5,0,гражданский брак,1,0,255763,недвижимость
6,0,женат / замужем,0,0,240525,недвижимость
7,0,женат / замужем,0,0,135823,образование
8,2,гражданский брак,1,0,95856,свадьба
9,0,женат / замужем,0,0,144425,недвижимость


**Вывод**

Первоначально визуально просмотрел уникальные значения столбца 'purpose'. Увидел, что основных категорий 4: операции с недвижимостью, с автомобилями, с обучением и со свадьбами. С помощью лемматизации были обновлены значения столбца 'purpose'. Значения вышеупомянутого столбца были заменены на один из 4х вариантов.

### Категоризация данных

In [199]:
#Создание таблицы для категории 'children'
df_children = solvency_table[['children', 'debt']] 
df_children = df_children.groupby('children').sum().sort_values(by = 'children', ascending = True)
df_children['total_debt'] = solvency_table.groupby('children')['debt'].count().sort_values()
df_children['debt/total_debt'] = (df_children['debt'] / df_children['total_debt']) * 100

In [200]:
#Создание таблицы для категории 'family_status'
df_family_status = solvency_table[['family_status', 'debt']] 
df_family_status = df_family_status.groupby('family_status').sum().sort_values(by = 'family_status', ascending = True)
df_family_status['total_debt'] = solvency_table.groupby('family_status')['debt'].count().sort_values()
df_family_status['debt/total_debt'] = (df_family_status['debt'] / df_family_status['total_debt']) * 100

<div class="alert alert-danger">

**Комментарий ревьюера:** Ты используешь верный инструмент для расчетов, но все таки только по количеству задолженностей нельзя сделать верный вывод. Нам следует считать процент задолженностей от общего числа заемов. Чтобы провести такие расчеты, у нас должна быть таблица из четырех столбцов: 1) группа/категория заемщиков, 2) количество задолженностей, 3) общее числой займов, 4) процент задолженностей от общего числа займов. И эти расчеты следует перенести в соответствующий раздел с ответами на вопросы. Уверен у тебя все получится :)
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Изменил таблицу. Добавил новые столбцы. Выводы поменялись.</div>

In [201]:
#Создание таблицы для категории 'total_income_lvl'
def salary_lvl(salary):
    if salary < 200000:
        return 'до 200 000'
    if 200000 <= salary < 400000:
        return 'от 200 000 до 400 000'
    if 400000 <= salary < 600000:
        return 'от 400 000 до 600 000'
    if 600000 <= salary < 800000:
        return 'от 600 000 до 800 000'
    if 800000 <= salary < 900000:
        return 'от 800 000 до 1 000 000'
    if salary > 1000000:
        return 'больше 1 000 000'

solvency_table['total_income_lvl'] = solvency_table['total_income'].apply(salary_lvl)
df_total_income_lvl = solvency_table[['total_income_lvl', 'debt']] 
df_total_income_lvl = df_total_income_lvl.groupby('total_income_lvl').sum()
df_total_income_lvl['total_debt'] = solvency_table.groupby('total_income_lvl')['debt'].count()
df_total_income_lvl['debt/total_debt'] = (df_total_income_lvl['debt'] / df_total_income_lvl['total_debt']) * 100

<div class="alert alert-warning">

**Комментарий ревьюера:** Ты молодец, что разделил на такое количество категорий, категоризация определенно нужна для данного столбца. Но у нас не так много данных, чтобы делать такую детальную разбивку, попробуй сделать около 5-6 категорий, так у тебя будет больше информации по каждой группе.
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Исправлено</div>

In [202]:
#Создание таблицы для категории 'purpose'
df_purpose = solvency_table[['purpose', 'debt']] 
df_purpose = df_purpose.groupby('purpose').sum().sort_values(by = 'purpose', ascending = True)
df_purpose['total_debt'] = solvency_table.groupby('purpose')['debt'].count().sort_values()
df_purpose['debt/total_debt'] = (df_purpose['debt'] / df_purpose['total_debt']) * 100

***Общий вывод***

Было произведено разделение на категории. Бысли созданны отдельные таблицы по каждой категории. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [203]:
df_children.sort_values(by = 'debt/total_debt', ascending = False)

,debt,total_debt,debt/total_debt
children,,,
20,8,76,10.526316
4,4,40,10.000000
2,192,1927,9.963674
1,436,4520,9.646018
3,27,325,8.307692
0,1017,12935,7.862389
5,0,9,0.000000


В категории 'children' не видна ярко выраженная зависимость процента возврата кредита от числа детей. Можно сказать, что люди, у которых нет детей, реже имеют задолжности, чем те, у кого дети есть. Для количества детей равного 5 мало данных, поэтому нельзя сделать никаких выводов для жтой позиции.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [204]:
df_family_status.sort_values(by = 'debt/total_debt', ascending = False)

,debt,total_debt,debt/total_debt
family_status,,,
Не женат / не замужем,272,2616,10.397554
гражданский брак,367,3845,9.544863
женат / замужем,898,11315,7.936368
в разводе,85,1149,7.397737
вдовец / вдова,62,907,6.835722


Видна зависимость задолжностей от семейного положения. Люди, которые не состоят в браке, чаще имеют задолжности. Меньше всего задолжностей имеют вдовцы и вдовы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [205]:
df_total_income_lvl.sort_values(by = 'debt/total_debt', ascending = False)

,debt,total_debt,debt/total_debt
total_income_lvl,,,
от 800 000 до 1 000 000,2,12,16.666667
до 200 000,1326,14766,8.980089
больше 1 000 000,2,25,8.000000
от 200 000 до 400 000,327,4538,7.205818
от 600 000 до 800 000,4,66,6.060606
от 400 000 до 600 000,23,419,5.489260


Наблюдается зависимость наличия задолжностей от уровня ежемесячного дохода. Чаще имеют задолжности люди с доходом от 800 000 р до 1 000 000 р. Реже всего имеют задолжности люди с доходом от 400 000 до 600 000.

- Как разные цели кредита влияют на его возврат в срок?

In [206]:
df_purpose.sort_values(by = 'debt/total_debt', ascending = False)

,debt,total_debt,debt/total_debt
purpose,,,
автомобиль,394,4012,9.820538
образование,358,3712,9.644397
свадьба,166,2117,7.841285
недвижимость,766,9991,7.666900


Видна зависимость между целью кредита и количеством задолжностей. Чаще имеют задолжности на цели образования и покупку автомобиля.

<div class="alert alert-danger">

**Комментарий ревьюера:** Как я уже писал выше, сюда лучше перенести расчеты и провести их по новому. Также, возможно, ты решишь пересмотреть свои ответы на вопросы после измененных расчетов.
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Исправлено. Вывод по каждому вопросу пересмотрен.</div>

## Шаг 4. Общий вывод

В данной работе были пройдены следающие этапы:
- открытие исходной таблицы
- визуальный анализ столбцов и строк
- создание таблицы на основе исходной, но только с теми столбцами, которые нужны
- в столбце 'total_income' пропуски были заменены медианой
- в столбце 'children' отрицательные значения были взяты по модулю
- в таблице было обнаружено довольно большое количество дубликатов, которые в последствии были удалены
- были восстановленны индексы таблицы, после удаления дубликатов
- с помощью лемматизации были заменены словосочетания из столбца 'purpose' на удобные тэги
- были выведены таблицы, где видна зависимость задолжностей от различных показателей
- по каждой таблице были сделаны выводы

В итоге можно сказать, что зависимость наличия задолжностей от от различных факторов наблюдается. В некоторых случаях она наблюдается ярче (уровень заработка и цель кредита), в некоторых случаях зависимость видна не так ярко (наличие/количество детей, семейное положение.

Я бы рекомендовал при принятии решения о выдаче кредита смотреть на данные показатели в совокупности. Тогда можно было бы точнее предположить вероятность возникновения задолжностей.

<div class="alert alert-danger">

**Комментарий ревьюера:** Какие конкретные рекомендации ты можешь дать по основной бизнес задаче? Что ты узнал отвечая на вопросы?
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Выполнено.</div>

<div class="alert alert-danger">

**Вывод:** Ты проделал большую работу и у тебя получился хороший проект! Но давай его немного улучшим и доработаем. Для этого тебе осталось:
* Вывести общую информацию по таблице на этапе ознакомления с данными
* Привести вещественные типы данных к целочисленным
* Пересмотреть расчеты по таблицам и перенести их в блок ответов на вопросы
* Пересмотреть ответы на вопросы, если потребуется
* Добавить рекомендацию в общий вывод
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Выполнено </div>

<div class="alert alert-success">

**Вывод v2:** Спасибо, что учел все мои комментарии! Было приятно с тобой поработать! С радостью сообщаю, что твой проект принят! Поздравляю с завершением первого проекта и удачи на следующем спринте! :) 
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

P.S. Заранее извиняюсь, если мой код был непонятен и были допущены грубые ошибки.

P.S.S. Не придумал, где можно использовать конструкцию try-except